In [1]:
import pathlib

import pandas as pd

In [2]:
path_to_data = pathlib.Path("../../data/images").resolve(strict=True)

In [3]:
# get all files in the data directory
files = path_to_data.glob("*")

In [4]:
metadata_dict = {
    "condition": [],
    "cell_number": [],
    "mitochondria_number": [],
    "file_type": [],
    "file_name": [],
    "file_path": [],
}

In [5]:
for file in files:
    metadata_dict["condition"].append(file.stem.split("_")[0])
    metadata_dict["cell_number"].append(file.stem.split("_")[2])
    metadata_dict["mitochondria_number"].append(file.stem.split("_")[3])
    metadata_dict["file_type"].append(file.stem.split("_")[4])
    metadata_dict["file_name"].append(file.stem)
    metadata_dict["file_path"].append(file)

In [6]:
# make a dataframe from the dictionary
metadata_df = pd.DataFrame(metadata_dict)
print(metadata_df.shape)
metadata_df.head()

(1840, 6)


,condition,cell_number,mitochondria_number,file_type,file_name,file_path
0,images,cell,004,0001,images_shCtrl_cell_004_0001_lamellar_cristae_imgs,/home/lippincm/Documents/4TB/data/ultrastrucur...
1,images,cell,003,0010,images_shCtrl_cell_003_0010_tubular_cristae_imgs,/home/lippincm/Documents/4TB/data/ultrastrucur...
2,images,cell,001,0134,images_shOPA1_cell_001_0134_ori_imgs,/home/lippincm/Documents/4TB/data/ultrastrucur...
3,images,cell,002,0097,images_shOPA1_cell_002_0097_mito_imgs,/home/lippincm/Documents/4TB/data/ultrastrucur...
4,images,cell,001,0091,images_shOPA1_cell_001_0091_mito_imgs,/home/lippincm/Documents/4TB/data/ultrastrucur...


In [7]:
# save the metadata to a csv file
metadata_df.to_csv("../../data/metadata.csv", index=False)